# LangChain 1.0 Multimodal in Databricks

Ref:
https://docs.langchain.com/oss/python/langchain/messages#multimodal

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 databricks-langchain mlflow

%restart_python

## モデルとの接続

In [0]:
from langchain_openai import ChatOpenAI
import mlflow

mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
model = ChatOpenAI(
    model="databricks-gemma-3-12b",
    base_url=creds.host + "/serving-endpoints",
    api_key=creds.token,
)


## 単純なエージェントの作成と実行

In [0]:
import base64

with open("sample.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")

In [0]:
from langchain.messages import HumanMessage

human_message = HumanMessage(
    content=[
        {"type": "text", "text": "この画像を説明して"},
        {
            "type": "image",
            "base64": image_data,
            "mime_type": "image/png",
        },
    ],
)

In [0]:
from langchain.agents import create_agent
from langchain_core.messages import AIMessageChunk
from pprint import pprint

agent = create_agent(
    model=model,
    # tools=[get_weather],
    tools=[],
    system_prompt="You are a helpful assistant",
)

input = {"messages": [human_message]}
for event in agent.stream(input, stream_mode="messages"):
    for chunk in event:
        if isinstance(chunk, AIMessageChunk):
            print(chunk.content, end="", flush=True)
